In [1]:
import tensorflow as tf

In [2]:
learning_rate = 1e-4
keep_prob_rate = 0.7
max_epoch = 3
batch_size = 50

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [4]:
def conv_layer(x, filters, kernel_size, activation='relu'):
    x = tf.keras.layers.Conv2D(filters=filters,
                               kernel_size=kernel_size,
                               padding='same',
                               activation=activation)(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    return x

In [5]:
inputs = tf.keras.Input(shape=(28, 28, 1))
x = inputs
x = conv_layer(x, 32, kernel_size=(7, 7))
x = conv_layer(x, 64, kernel_size=(5, 5))
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(keep_prob_rate)(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [6]:
class PrintTestAccuracy(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch+1) % 100 == 0:
            test_acc = model.evaluate(x_test, y_test, verbose=0)[1]
            print(f'Epoch {epoch+1}, Test accuracy: {test_acc*100:.6f}%')

In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [PrintTestAccuracy()]

history = model.fit(x_train, y_train, epochs=max_epoch, batch_size=batch_size, validation_split=0.1, callbacks=callbacks)

Epoch 1/3
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 65s 59ms/step - accuracy: 0.7726 - loss: 0.7623 - val_accuracy: 0.9762 - val_loss: 0.0855
Epoch 2/3
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 62s 57ms/step - accuracy: 0.9655 - loss: 0.1190 - val_accuracy: 0.9832 - val_loss: 0.0592
Epoch 3/3
1080/1080 ━━━━━━━━━━━━━━━━━━━━ 62s 58ms/step - accuracy: 0.9751 - loss: 0.0804 - val_accuracy: 0.9848 - val_loss: 0.0499


In [8]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test accuracy:", test_acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9810 - loss: 0.0558
Test accuracy: 0.9836999773979187
